## Colab과 BigQuery 연동

- 1. 구글 계정 인증

In [2]:
from google.colab import auth
auth.authenticate_user()

- 2. BigQuery 클라이언트 설정

In [3]:
from google.cloud import bigquery
client = bigquery.Client(project='e-commerce-project-438001')

## 1. EDA

### Customer

- customer 테이블 데이터 확인

In [4]:
query = '''
SELECT *
FROM e_commerce_project.customer;
'''
df = client.query(query).to_dataframe()
df.head()

,customer_id,sex,local,period
0,USER_0832,female,Chicago,2
1,USER_1311,male,Chicago,2
2,USER_0270,male,Chicago,2
3,USER_0732,female,Chicago,2
4,USER_1026,female,Chicago,2


- 성별별 유저수 및 비율 확인
  - 여성:924명(64%), 남성:534명(36%) 여성 유저수가 더 많음

In [5]:
query = '''
SELECT COUNT(CASE WHEN sex = 'female' THEN customer_id END) AS cnt_female
      , COUNT(CASE WHEN sex = 'male' THEN customer_id END) AS cnt_male
      , ROUND(COUNT(CASE WHEN sex = 'female' THEN customer_id END) / COUNT(*), 2) AS pct_female
      , ROUND(COUNT(CASE WHEN sex = 'male' THEN customer_id END) / COUNT(*), 2) AS pct_male
FROM e_commerce_project.customer;
'''
df = client.query(query).to_dataframe()
df.head()

,cnt_female,cnt_male,pct_female,pct_male
0,934,534,0.64,0.36


- 가입긴간별 유저수 분포 확인
  - 전반적으로 가입기간별로 총 유저에 큰 차이는 없음
  - 가입기간이 6, 12, 36개월인 유저에서 남성 유저가 여성 유저보다 많음

In [6]:
query = '''
SELECT period
      , COUNT(customer_id) AS total_user
      , COUNT(CASE WHEN sex = 'female' THEN customer_id END) AS cnt_female
      , COUNT(CASE WHEN sex = 'male' THEN customer_id END) AS cnt_male
FROM e_commerce_project.customer
GROUP BY period;
'''
df = client.query(query).to_dataframe()
df.head()

,period,total_user,cnt_female,cnt_male
0,2,26,17,9
1,3,23,16,7
2,4,38,28,10
3,5,30,24,6
4,6,36,17,19


- 지역별 유저수 분포 확인
  - 캘리포니아>시카고>뉴욕>뉴저지>워싱턴DC 유저 수가 많음

In [7]:
query = '''
SELECT local
      , COUNT(customer_id) AS total_user
      , COUNT(CASE WHEN sex = 'female' THEN customer_id END) AS cnt_female
      , COUNT(CASE WHEN sex = 'male' THEN customer_id END) AS cnt_male
FROM e_commerce_project.customer
GROUP BY local
ORDER BY total_user DESC;
'''
df = client.query(query).to_dataframe()
df.head()

,local,total_user,cnt_female,cnt_male
0,California,464,299,165
1,Chicago,456,285,171
2,New York,324,213,111
3,New Jersey,149,97,52
4,Washington DC,75,40,35


- 가입기간별 지역 분포 확인

In [8]:
query = '''
SELECT period
      , COUNT(customer_id) AS total_user
      , COUNT(CASE WHEN local = 'California' THEN customer_id END) AS cnt_california
      , COUNT(CASE WHEN local = 'Chicago' THEN customer_id END) AS cnt_chicago
      , COUNT(CASE WHEN local = 'New York' THEN customer_id END) AS cnt_newyork
      , COUNT(CASE WHEN local = 'New Jersey' THEN customer_id END) AS cnt_newjersey
      , COUNT(CASE WHEN local = 'Washington DC' THEN customer_id END) AS cnt_washingtiondc
FROM e_commerce_project.customer
GROUP BY period
ORDER BY period;
'''
df = client.query(query).to_dataframe()
df.head()

,period,total_user,cnt_california,cnt_chicago,cnt_newyork,cnt_newjersey,cnt_washingtiondc
0,2,26,7,7,6,5,1
1,3,23,5,9,6,2,1
2,4,38,6,14,7,7,4
3,5,30,13,5,7,2,3
4,6,36,10,12,8,2,4


### Discount

- discount 테이블 데이터 확인

In [9]:
query = '''
SELECT *
FROM e_commerce_project.discount;
'''
df = client.query(query).to_dataframe()
df.head()

,month,category,coupon_code,discount_rate,month_numeric
0,Jan,Apparel,SALE10,10,01
1,Jan,Bottles,BT10,10,01
2,Jan,Waze,WEMP10,10,01
3,Jan,Lifestyle,EXTRA10,10,01
4,Jan,Bags,AIO10,10,01


- 각 컬럼별 유니크한 값 확인

In [10]:
query = '''
SELECT COUNT(*) AS cnt_total
      , COUNT(DISTINCT month) AS cnt_month
      , COUNT(DISTINCT category) AS cnt_category
      , COUNT(DISTINCT coupon_code) AS cnt_coupon_code
      , COUNT(DISTINCT discount_rate) AS cnt_discount_rate
FROM e_commerce_project.discount;
'''
df = client.query(query).to_dataframe()
df.head()

,cnt_total,cnt_month,cnt_category,cnt_coupon_code,cnt_discount_rate
0,204,12,17,48,3


- 할인율에 무슨 값이 있는지 확인
 - 10, 20, 30만 있음

In [11]:
query = '''
SELECT DISTINCT discount_rate
FROM e_commerce_project.discount;
'''
df = client.query(query).to_dataframe()
df.head()

,discount_rate
0,10
1,20
2,30


- 월별 할인율 확인
  - 1,4,7,10월은 10%
  - 2,5,8,11월은 20%
  - 3,6,9,12월은 30%로 할인율 고정됨

In [12]:
query = '''
SELECT DISTINCT month
      , discount_rate
FROM e_commerce_project.discount;
'''
df = client.query(query).to_dataframe()
df

,month,discount_rate
0,Jan,10
1,Oct,10
2,Apr,10
3,Jul,10
4,Feb,20
5,Nov,20
6,May,20
7,Aug,20
8,Mar,30
9,Dec,30


- 카테고리별 쿠폰 코드 확인
  - Drinkware, Lifestyle의 쿠폰 코드가 'EXTRA**'로 동일한 것을 제외하면 모두 다름

In [13]:
query = '''
SELECT DISTINCT category
      , coupon_code
FROM e_commerce_project.discount
ORDER BY category;
'''
df = client.query(query).to_dataframe()
df

,category,coupon_code
0,Accessories,ACC10
1,Accessories,ACC20
2,Accessories,ACC30
3,Android,AND10
4,Android,AND20
5,Android,AND30
6,Apparel,SALE10
7,Apparel,SALE20
8,Apparel,SALE30
9,Bags,AIO10


### Marketing

- marketing 테이블 데이터 확인

In [14]:
query = '''
SELECT *
FROM e_commerce_project.marketing;
'''
df = client.query(query).to_dataframe()
df.head()

,date,offline_cost,online_cost
0,2019-01-08,5000,2551.38
1,2019-01-09,5000,3287.83
2,2019-01-10,5000,515.44
3,2019-01-11,5000,1757.58
4,2019-01-12,5000,2435.97


- 각 컬럼별 유니크 값 확인
  - 오프라인 마케팅 비용은 정해진 비용이 있는 거 같음
  - 온라인 마케팅 비용은 매일 변함

In [15]:
query = '''
SELECT COUNT(*) AS cnt_total
      , COUNT(DISTINCT date) AS cnt_date
      , COUNT(DISTINCT offline_cost) AS cnt_offline
      , COUNT(DISTINCT online_cost) AS cnt_online
FROM e_commerce_project.marketing;
'''
df = client.query(query).to_dataframe()
df.head()

,cnt_total,cnt_date,cnt_offline,cnt_online
0,365,365,11,365


- 일별로 정렬해서 비용 비교해보기
  - 오프라인 마케팅 비용은 주단위로 주기를 가짐

In [16]:
query = '''
SELECT *
FROM e_commerce_project.marketing
ORDER BY date;
'''
df = client.query(query).to_dataframe()
df

,date,offline_cost,online_cost
0,2019-01-01,4500,2424.50
1,2019-01-02,4500,3480.36
2,2019-01-03,4500,1576.38
3,2019-01-04,4500,2928.55
4,2019-01-05,4500,4055.30
...,...,...,...
360,2019-12-27,4000,3396.87
361,2019-12-28,4000,3246.84
362,2019-12-29,4000,2546.58
363,2019-12-30,4000,674.31


- 오프라인 마케팅 비용 값 확인
  - 500~5000까지 500씩 증가, 700 제외

In [17]:
query = '''
SELECT DISTINCT offline_cost
FROM e_commerce_project.marketing
ORDER BY offline_cost;
'''
df = client.query(query).to_dataframe()
df

,offline_cost
0,500
1,700
2,1000
3,1500
4,2000
5,2500
6,3000
7,3500
8,4000
9,4500


### Onlinesales2

- onlinesales2 테이블 데이터 확인

In [18]:
query = '''
SELECT *
FROM e_commerce_project.onlinesales2;
'''
df = client.query(query).to_dataframe()
df.head()

,customer_id,new_order_id,date,product_id,category,quantity,avg_cost,shipping_fee,coupon_status,month
0,USER_0190,Transaction_0005,2019-01-01,Product_0310,Apparel,1,14.02,6.5,Used,01
1,USER_0190,Transaction_0005,2019-01-01,Product_0486,Apparel,1,14.02,6.5,Clicked,01
2,USER_0190,Transaction_0006,2019-01-01,Product_0906,Lifestyle,1,9.27,6.5,Clicked,01
3,USER_0190,Transaction_0006,2019-01-01,Product_0949,Office,3,1.55,6.5,Clicked,01
4,USER_0190,Transaction_0006,2019-01-01,Product_0951,Office,1,3.08,6.5,Used,01


- 각 컬럼별 유니크한 값 확인

In [19]:
query = '''
SELECT COUNT(*) AS cnt_total
      , COUNT(DISTINCT customer_id) AS cnt_customer_id
      , COUNT(DISTINCT new_order_id) AS cnt_new_order_id
      , COUNT(DISTINCT date) AS cnt_date
      , COUNT(DISTINCT product_id) AS cnt_product_id
      , COUNT(DISTINCT category) AS cnt_category
      , COUNT(DISTINCT quantity) AS cnt_quantity
      , COUNT(DISTINCT avg_cost) AS cnt_avg_cost
      , COUNT(DISTINCT shipping_fee) AS cnt_shipping_fee
      , COUNT(DISTINCT coupon_status) AS cnt_coupon_status
FROM e_commerce_project.onlinesales2;
'''
df = client.query(query).to_dataframe()
df.head()

,cnt_total,cnt_customer_id,cnt_new_order_id,cnt_date,cnt_product_id,cnt_category,cnt_quantity,cnt_avg_cost,cnt_shipping_fee,cnt_coupon_status
0,52924,1468,26631,365,1145,20,151,546,267,3


- onlinesales2 테이블의 category와 discount, tax 테이블의 category와 차이점 확인
  - onlinesales2, tax 테이블에는 discount 테이블에는 없는 Backpacks, Fun, Google, More Bags 카테고리가 있습니다.
  - discount 테이블에는 onlinesales2, tax 테이블에는 없는 Notebooks 카테고리가 있습니다.
- 추후 분석을 수행할 때는 Onlinesales2 테이블을 기준으로 LEFT JOIN을 활용하여 Notebooks 카테고리는 사용하지 않습니다.

In [20]:
query = '''
SELECT DISTINCT category
FROM e_commerce_project.onlinesales2
ORDER BY category;
'''
df = client.query(query).to_dataframe()
df.head()

,category
0,Accessories
1,Android
2,Apparel
3,Backpacks
4,Bags


In [21]:
query = '''
SELECT DISTINCT category
FROM e_commerce_project.discount
ORDER BY category;
'''
df = client.query(query).to_dataframe()
df.head()

,category
0,Accessories
1,Android
2,Apparel
3,Bags
4,Bottles


In [22]:
query = '''
SELECT DISTINCT category
FROM e_commerce_project.tax
ORDER BY category;
'''
df = client.query(query).to_dataframe()
df.head()

,category
0,Accessories
1,Android
2,Apparel
3,Backpacks
4,Bags


- 쿠폰 상태 및 상태별 주문수 확인
  - Used: 17904 / Clicked: 26926 / Not Used: 8094
  - 쿠폰 사용이 적은 것을 확인했습니다.

In [23]:
query = '''
SELECT coupon_status
      , COUNT(customer_id)
FROM e_commerce_project.onlinesales2
GROUP BY coupon_status;
'''
df = client.query(query).to_dataframe()
df.head()

,coupon_status,f0_
0,Used,17904
1,Clicked,26926
2,Not Used,8094


- 할인율, 세율, 배송비를 고려하지 않은 총 매출액 확인 (quantity * avg_cost)
  - $ 4,670,794.62

In [24]:
query = '''
SELECT ROUND(SUM(quantity * avg_cost), 2) AS total_sales
FROM e_commerce_project.onlinesales2;
'''
df = client.query(query).to_dataframe()
df.head()

,total_sales
0,4670794.62


- 판매 기간 확인
  - 2019-01-01 ~ 2019-12-31

In [25]:
query = '''
SELECT MIN(date)
      , MAX(date)
FROM e_commerce_project.onlinesales2;
'''
df = client.query(query).to_dataframe()
df.head()

,f0_,f1_
0,2019-01-01,2019-12-31


- 일자별 매출 추이 확인

In [29]:
query = '''
SELECT date
      , ROUND(SUM(quantity * avg_cost), 2) AS daily_sales
FROM e_commerce_project.onlinesales2
GROUP BY date;
'''
df = client.query(query).to_dataframe()
df

,date,daily_sales
0,2019-01-01,7380.49
1,2019-01-02,13438.47
2,2019-01-03,23834.21
3,2019-01-04,16603.41
4,2019-01-05,17373.19
...,...,...
360,2019-12-27,13661.59
361,2019-12-28,10994.42
362,2019-12-29,12011.27
363,2019-12-30,8068.47


- 전체 고객의 평균 주문 주기 확인

In [28]:
query = '''
WITH customer_orders AS(
  SELECT customer_id
        , date
        ,ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY date) AS order_num
  FROM e_commerce_project.onlinesales2
  GROUP BY customer_id, date
), purchase_intervals AS(
  SELECT c1.customer_id
        , DATE_DIFF(c2.date, c1.date, DAY) AS purchase_interval
  FROM customer_orders AS c1
    LEFT JOIN customer_orders AS c2 ON c1.customer_id = c2.customer_id AND c2.order_num = c1.order_num + 1
), purchase_interval_customers AS(
  SELECT customer_id
        , AVG(purchase_interval) AS avg_purchase_interval
  FROM purchase_intervals
  GROUP BY customer_id
  HAVING COUNT(*) >= 2
)
SELECT AVG(avg_purchase_interval) AS total_avg_purchase_interval
FROM purchase_interval_customers;
'''
df = client.query(query).to_dataframe()
df.head()

,total_avg_purchase_interval
0,64.925683


### Tax

- tax 테이블 데이터 확인

In [31]:
query = '''
SELECT *
FROM e_commerce_project.tax;
'''
df = client.query(query).to_dataframe()
df

,category,gst
0,Apparel,0.18
1,Bags,0.18
2,Drinkware,0.18
3,Lifestyle,0.18
4,Waze,0.18
5,Fun,0.18
6,More Bags,0.18
7,Notebooks & Journals,0.05
8,Headgear,0.05
9,Bottles,0.05


## 문제해결

### 문제상황1
- order_id 하나당 customer_id가 하나여야 하는데 그러지 못한 order_id들이 1319개 있음


In [33]:
query = '''
SELECT order_id
      , COUNT(DISTINCT customer_id) AS cnts
FROM e_commerce_project.onlinesales
GROUP BY order_id
HAVING cnts > 1
ORDER BY order_id;
'''
df = client.query(query).to_dataframe()
df

,order_id,cnts
0,Transaction_0003,2
1,Transaction_0013,3
2,Transaction_0050,2
3,Transaction_0072,3
4,Transaction_0076,3
...,...,...
1314,Transaction_9967,3
1315,Transaction_9978,2
1316,Transaction_9985,2
1317,Transaction_9996,2


- 해결방법1: 엑셀 수식을 사용해 중복이 있는 order_id에 대해서는 customer_id의 뒤의 5자리를 order_id에 결합하여 서로 구분할 수 있도록 함
  - 수정후 onlinesales2라는 테이블로 새로 업로드

- 해결 방법2: SQL쿼리로 중복이 있는 order_id에 대해서는 customer_id의 뒤의 5자리를 order_id에 결합하여 new_order_id 컬럼을 생성하여 서로 구분할 수 있도록 함
  - 수정후 onlinesales2라는 테이블로 새로 업로드

In [34]:
query = '''
 WITH double_order AS(
  SELECT order_id
        , COUNT(DISTINCT customer_id) AS cnts
  FROM e_commerce_project.onlinesales
  GROUP BY order_id
)
  SELECT o.customer_id
        , CASE
              WHEN do.cnts > 1 THEN CONCAT(o.order_id, RIGHT(o.customer_id, 5))
              ELSE o.order_id
        END AS new_order_id
        , o.date
        , o.product_id
        , o.category
        , o.quantity
        , o.avg_cost
        , o.shipping_fee
        , o.coupon_status
  FROM e_commerce_project.onlinesales AS o
    LEFT JOIN double_order AS do ON o.order_id = do.order_id;
'''
df = client.query(query).to_dataframe()
df.head()

,customer_id,new_order_id,date,product_id,category,quantity,avg_cost,shipping_fee,coupon_status
0,USER_1358,Transaction_0003_1358,2019-01-01,Product_0848,Bags,1,16.50,6.5,Used
1,USER_1358,Transaction_0003_1358,2019-01-01,Product_0898,Drinkware,5,9.27,6.5,Used
2,USER_0190,Transaction_0003_0190,2019-01-01,Product_1025,Apparel,5,17.53,6.5,Used
3,USER_0190,Transaction_0005,2019-01-01,Product_0310,Apparel,1,14.02,6.5,Used
4,USER_0190,Transaction_0006,2019-01-01,Product_0951,Office,1,3.08,6.5,Used


### 문제상황2
- onlinesales2와 discount 테이블을 조인할 컬럼이 없어 오류 발생(category로 조인하면 같은 onlinesales2 테이블 한 행당 discount 테이블의 month 12개가 조인되어 month로 조인 필요)

- 1단계: discount 테이블의 month를 숫자 형태로 변환한 month_numeric 컬럼 추가 생성

In [36]:
query = '''
SELECT *,
  CASE
    WHEN month = 'Jan' THEN '01'
    WHEN month = 'Feb' THEN '02'
    WHEN month = 'Mar' THEN '03'
    WHEN month = 'Apr' THEN '04'
    WHEN month = 'May' THEN '05'
    WHEN month = 'Jun' THEN '06'
    WHEN month = 'Jul' THEN '07'
    WHEN month = 'Aug' THEN '08'
    WHEN month = 'Sep' THEN '09'
    WHEN month = 'Oct' THEN '10'
    WHEN month = 'Nov' THEN '11'
    WHEN month = 'Dec' THEN '12'
  END AS month_numeric
FROM e_commerce_project.discount;
'''
df = client.query(query).to_dataframe()
df

,month,category,coupon_code,discount_rate,month_numeric,month_numeric_1
0,Jan,Apparel,SALE10,10,01,01
1,Jan,Bottles,BT10,10,01,01
2,Jan,Waze,WEMP10,10,01,01
3,Jan,Lifestyle,EXTRA10,10,01,01
4,Jan,Bags,AIO10,10,01,01
...,...,...,...,...,...,...
199,Sep,Nest,NE30,30,09,09
200,Sep,Bags,AIO30,30,09,09
201,Sep,Waze,WEMP30,30,09,09
202,Sep,Housewares,HOU30,30,09,09


- 2단계: onlinesales2 테이블의 date 컬럼을 기준으로 month 컬럼 생성

In [37]:
query = '''
SELECT *
      , FORMAT_DATE('%m', date) AS month
FROM e_commerce_project.onlinesales2;
'''
df = client.query(query).to_dataframe()
df.head()

,customer_id,new_order_id,date,product_id,category,quantity,avg_cost,shipping_fee,coupon_status,month,month_1
0,USER_0190,Transaction_0005,2019-01-01,Product_0310,Apparel,1,14.02,6.5,Used,01,01
1,USER_0190,Transaction_0005,2019-01-01,Product_0486,Apparel,1,14.02,6.5,Clicked,01,01
2,USER_0190,Transaction_0006,2019-01-01,Product_0906,Lifestyle,1,9.27,6.5,Clicked,01,01
3,USER_0190,Transaction_0006,2019-01-01,Product_0949,Office,3,1.55,6.5,Clicked,01,01
4,USER_0190,Transaction_0006,2019-01-01,Product_0951,Office,1,3.08,6.5,Used,01,01


- 3단계: 조인 확인

In [38]:
query = '''
SELECT *
FROM e_commerce_project.onlinesales2 AS o
  INNER JOIN e_commerce_project.discount AS d ON o.month = d.month_numeric
  AND o.category = d.category;
'''
df = client.query(query).to_dataframe()
df.head()

,customer_id,new_order_id,date,product_id,category,quantity,avg_cost,shipping_fee,coupon_status,month,month_1,category_1,coupon_code,discount_rate,month_numeric
0,USER_0190,Transaction_0005,2019-01-01,Product_0310,Apparel,1,14.02,6.5,Used,01,Jan,Apparel,SALE10,10,01
1,USER_0190,Transaction_0005,2019-01-01,Product_0486,Apparel,1,14.02,6.5,Clicked,01,Jan,Apparel,SALE10,10,01
2,USER_0190,Transaction_0006,2019-01-01,Product_0906,Lifestyle,1,9.27,6.5,Clicked,01,Jan,Lifestyle,EXTRA10,10,01
3,USER_0190,Transaction_0006,2019-01-01,Product_0949,Office,3,1.55,6.5,Clicked,01,Jan,Office,OFF10,10,01
4,USER_0190,Transaction_0006,2019-01-01,Product_0951,Office,1,3.08,6.5,Used,01,Jan,Office,OFF10,10,01


## 2. RFM

### RFM 분류

- Recency: 전체 고객의 마지막 주문일인 '2019-12-31' 하루 뒤인 '2020-01-01'를 기준으로 각 고객별 마지막 주문일의 차이를 계산
- Frequency: 고객별 유니크한 new_order_id의 수를 집계
- Monetary:
  - onlinesales2 테이블에만 있는 category: quantity * avg_cost * (1-discount_rate/100) + quantity * avg_cost * (1-discount_rate/100) * gst + shipping_fee
  - `Clicked, Not Used`: quantity * avg_cost + quantity * avg_cost * gst + shipping_fee
  - `Used`: quantity * avg_cost * (1-discount_rate/100) + quantity * avg_cost * (1-discount_rate/100) * gst + shipping_fee
    - 배송비는 new_order_id 당 shipping_fee가 동일하여 한번만 더하여 계산


In [39]:
query = '''
WITH subtotal_sales AS (
  SELECT o.customer_id
        , o.new_order_id
        , SUM(CASE
              WHEN d.discount_rate IS NULL THEN o.quantity*o.avg_cost+o.quantity*o.avg_cost*t.gst
              WHEN o.coupon_status = 'Used' THEN o.quantity*o.avg_cost*(1-d.discount_rate/100)+o.quantity*o.avg_cost*(1-d.discount_rate/100)*t.gst
              WHEN o.coupon_status <> 'Used' THEN o.quantity*o.avg_cost+o.quantity*o.avg_cost*t.gst
        END) subtotal_sales
  FROM e_commerce_project.onlinesales2 AS o
    LEFT JOIN e_commerce_project.customer AS c ON o.customer_id = c.customer_id
    LEFT JOIN e_commerce_project.discount AS d ON o.month = d.month_numeric AND o.category = d.category
    LEFT JOIN e_commerce_project.tax AS t ON o.category = t.category
  GROUP BY o.customer_id, o.new_order_id
), shipping_fee AS (
  SELECT DISTINCT new_order_id
        , shipping_fee
  FROM e_commerce_project.onlinesales2
), total_sales AS(
  SELECT ss.customer_id
        , ss.new_order_id
        , ss.subtotal_sales + sf.shipping_fee AS total_sales
  FROM shipping_fee AS sf
    INNER JOIN subtotal_sales AS ss ON sf.new_order_id = ss.new_order_id
), monetary AS(
  SELECT ts.customer_id
        , ROUND(SUM(ts.total_sales), 2) AS Monetary
  FROM total_sales AS ts
GROUP BY customer_id
), rf AS(
  SELECT o.customer_id
      , DATE_DIFF('2020-01-01', MAX(o.date), DAY) AS Recency
      , COUNT(DISTINCT o.new_order_id) AS Frequency
FROM e_commerce_project.onlinesales2 AS o
GROUP BY o.customer_id
)
SELECT rf.customer_id
      , rf.Recency
      , rf.Frequency
      , m.Monetary
FROM monetary AS m
  LEFT JOIN rf AS rf ON m.customer_id = rf.customer_id
ORDER BY customer_id
'''
df = client.query(query).to_dataframe()
df.head()

,customer_id,Recency,Frequency,Monetary
0,USER_0000,108,1,99.98
1,USER_0001,60,31,15308.16
2,USER_0002,74,8,1573.48
3,USER_0003,18,11,1421.95
4,USER_0004,108,13,1524.43


### RFM 기준별로 분류

- 사분위수와 이상치 공식 (Q3 + 1.5 * IQR)을 활용한 고객 등급별 인원수 및 매출액
  - 고객 상태는 계절성 제품인 Apperal이 거래량이 많은 것과 주문횟수 2회이상인 고객들의 평균 구매주기가 약65일인 것을 고려하여 '2020-01-01'을 기준으로 3개월 이내를 활성 고객, 그 외는 이탈 고객으로 분류

In [42]:
query = '''
WITH rfm_criteria AS(
  SELECT PERCENTILE_CONT(Frequency, 0.25) OVER() AS frequency_q1    -- 5
    , PERCENTILE_CONT(Frequency, 0.5) OVER() AS frequency_q2        -- 11
    , PERCENTILE_CONT(Frequency, 0.75) OVER() AS frequency_q3       -- 23
    , PERCENTILE_CONT(Frequency, 0.75) OVER() + 1.5 * (PERCENTILE_CONT(Frequency, 0.75) OVER() - PERCENTILE_CONT(Frequency, 0.25) OVER()) AS frequency_upper_bound                                               -- 50
    , PERCENTILE_CONT(Monetary, 0.25) OVER() AS monetary_q1         -- 713.7175
    , PERCENTILE_CONT(Monetary, 0.5) OVER() AS monetary_q2          -- 1889.5550
    , PERCENTILE_CONT(Monetary, 0.75) OVER() AS monetary_q3         -- 4249.4350
    , PERCENTILE_CONT(Monetary, 0.75) OVER() + 1.5 * (PERCENTILE_CONT(Monetary, 0.75) OVER() - PERCENTILE_CONT(Monetary, 0.25) OVER()) AS monetary_upper_bound                                                -- 9611.21875
  FROM e_commerce_project.rfm
  LIMIT 1
), rfm_score AS(
  SELECT rfm.*
        , CASE
              WHEN rfm.Recency <= 92 THEN '활성 고객'
              ELSE '이탈 고객' END AS customer_status
        , CASE
              WHEN rfm.Frequency >= rfm_criteria.frequency_upper_bound THEN 5
              WHEN rfm.Frequency >= rfm_criteria.frequency_q3 THEN 4
              WHEN rfm.Frequency >= rfm_criteria.frequency_q2 THEN 3
              WHEN rfm.Frequency >= rfm_criteria.frequency_q1 THEN 2
              ELSE 1 END AS F_score
        , CASE
              WHEN rfm.Monetary >= rfm_criteria.monetary_upper_bound THEN 'Black'
              WHEN rfm.Monetary >= rfm_criteria.monetary_q3 THEN 'Emerald'
              WHEN rfm.Monetary >= rfm_criteria.monetary_q2 THEN 'Purple'
              WHEN rfm.Monetary >= rfm_criteria.monetary_q1 THEN 'Red'
              ELSE 'Green' END AS customer_grade
  FROM e_commerce_project.rfm
    CROSS JOIN rfm_criteria
)
SELECT customer_status
      , F_score
      , customer_grade
      , COUNT(customer_id) AS customer_cnts
      , ROUND(SUM(Monetary), 2) AS sales
FROM rfm_score
GROUP BY customer_status, F_score, customer_grade
ORDER BY CASE
            WHEN customer_grade = 'Black' THEN 1
            WHEN customer_grade = 'Emerald' THEN 2
            WHEN customer_grade = 'Purple' THEN 3
            WHEN customer_grade = 'Red' THEN 4
            WHEN customer_grade = 'Green' THEN 5
         END, customer_status DESC, F_score DESC;
'''
df = client.query(query).to_dataframe()
df

,customer_status,F_score,customer_grade,customer_cnts,sales
0,활성 고객,5,Black,56,1037260.78
1,활성 고객,4,Black,17,224958.28
2,이탈 고객,5,Black,18,357076.60
3,이탈 고객,4,Black,8,107688.14
4,이탈 고객,3,Black,1,10463.69
5,활성 고객,5,Emerald,8,70075.44
6,활성 고객,4,Emerald,98,633190.26
7,활성 고객,3,Emerald,23,122141.74
8,이탈 고객,5,Emerald,8,66751.91
9,이탈 고객,4,Emerald,115,714894.47


- 위 기준을 활용하여 고객별로 분류 후 테이블 생성

In [41]:
query = '''
WITH rfm_criteria AS(
  SELECT PERCENTILE_CONT(Frequency, 0.25) OVER() AS frequency_q1    -- 5
        , PERCENTILE_CONT(Frequency, 0.5) OVER() AS frequency_q2        -- 11
        , PERCENTILE_CONT(Frequency, 0.75) OVER() AS frequency_q3       -- 23
          , PERCENTILE_CONT(Frequency, 0.75) OVER() + 1.5 * (PERCENTILE_CONT(Frequency, 0.75) OVER() - PERCENTILE_CONT(Frequency, 0.25) OVER()) AS frequency_upper_bound                                                 -- 50
        , PERCENTILE_CONT(Monetary, 0.25) OVER() AS monetary_q1         -- 713.7175
        , PERCENTILE_CONT(Monetary, 0.5) OVER() AS monetary_q2          -- 1889.5550
        , PERCENTILE_CONT(Monetary, 0.75) OVER() AS monetary_q3         -- 4249.4350
        , PERCENTILE_CONT(Monetary, 0.75) OVER() + 1.5 * (PERCENTILE_CONT(Monetary, 0.75) OVER() - PERCENTILE_CONT(Monetary, 0.25) OVER()) AS monetary_upper_bound                                                  -- 9611.21875
  FROM e_commerce_project.rfm
  LIMIT 1
), rfm_score AS(
  SELECT rfm.*
        , CASE
              WHEN rfm.Recency <= 92 THEN '활성 고객'
              ELSE '이탈 고객' END AS customer_status
        , CASE
              WHEN rfm.Frequency >= rfm_criteria.frequency_upper_bound THEN 5
              WHEN rfm.Frequency >= rfm_criteria.frequency_q3 THEN 4
              WHEN rfm.Frequency >= rfm_criteria.frequency_q2 THEN 3
              WHEN rfm.Frequency >= rfm_criteria.frequency_q1 THEN 2
              ELSE 1 END AS F_score
        , CASE
              WHEN rfm.Monetary >= rfm_criteria.monetary_upper_bound THEN 'Black'
              WHEN rfm.Monetary >= rfm_criteria.monetary_q3 THEN 'Emerald'
              WHEN rfm.Monetary >= rfm_criteria.monetary_q2 THEN 'Purple'
              WHEN rfm.Monetary >= rfm_criteria.monetary_q1 THEN 'Red'
              ELSE 'Green' END AS M_grade
  FROM e_commerce_project.rfm
  CROSS JOIN rfm_criteria
)
SELECT *
FROM rfm_score
ORDER BY customer_id
'''
df = client.query(query).to_dataframe()
df.head()

,customer_id,Recency,Frequency,Monetary,customer_status,F_score,M_grade
0,USER_0000,108,1,99.98,이탈 고객,1,Green
1,USER_0001,60,31,15308.16,활성 고객,4,Black
2,USER_0002,74,8,1573.48,활성 고객,2,Red
3,USER_0003,18,11,1421.95,활성 고객,3,Red
4,USER_0004,108,13,1524.43,이탈 고객,3,Red
